In [10]:
import os
import requests
from dotenv import load_dotenv
from langchain_openai import ChatOpenAI # tend to use this whenever gemini runs into quota limit
from langchain_google_vertexai import ChatVertexAI # runs into quota limit sometimes
from langchain.pydantic_v1 import BaseModel, Field
from langchain.tools import BaseTool, StructuredTool, tool
from typing import Optional, Type
from langchain.callbacks.manager import (
    AsyncCallbackManagerForToolRun,
    CallbackManagerForToolRun,
)
from langchain.agents import AgentExecutor, create_tool_calling_agent
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.messages import SystemMessage
from langgraph.checkpoint import MemorySaver  # an in-memory checkpointer
from langgraph.prebuilt import create_react_agent
load_dotenv()

True

In [2]:
# For tracing and debugging
os.environ["LANGCHAIN_TRACING_V2"] = "true"

# credential.json is not required if you are working within your own GCP project's vertex AI workbench
os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = "/workspaces/Transcendent/fleet-anagram-244304-7dafcc771b2f.json"

# if you are using text embedding model from google
os.environ["GOOGLE_API_KEY"] = os.getenv("GOOGLE_API_KEY")

GITHUB_ACCESS_TOKEN = os.getenv("GITHUB_ACCESS_TOKEN")
LANGCHAIN_API_KEY = os.getenv("LANGCHAIN_API_KEY")
GEMINI_API_KEY = os.getenv("GEMINI_API_KEY")
TAVILY_API_KEY = os.getenv("TAVILY_API_KEY")
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")

In [26]:
# Let's define a custom tool by subclassing the BaseTool class (from langchain)

class GithubUserCredentials(BaseModel):
    username: str = Field(description="Github username")
    github_access_token: str = Field(description="Github access token")
    
class GithubActivityTool(BaseTool):
    name = "Github_Activity_Tool"
    description = "useful for when you need to track user's github activity / events"
    args_schema: Type[BaseModel] = GithubUserCredentials

    def _run(self, username, github_access_token):
        url = f"https://api.github.com/users/{username}/events"
        headers = {"Authorization": f"token {github_access_token}"}

        response = requests.get(url, headers=headers)
        response.raise_for_status()  # Raise an exception for HTTP errors

        events_data = response.json()
        return events_data

In [27]:
github_event_search = GithubActivityTool()
print(github_event_search.name)
print(github_event_search.description)
print(github_event_search.args)

Github_Activity_Tool
useful for when you need to track user's github activity / events
{'username': {'title': 'Username', 'description': 'Github username', 'type': 'string'}, 'github_access_token': {'title': 'Github Access Token', 'description': 'Github access token', 'type': 'string'}}


In [28]:
llm = ChatOpenAI(model="gpt-4o")
tools = [GithubActivityTool()]

In [33]:
system_message = "You are a helpful assistant that specializes in tracking user's recent github activity."

memory = MemorySaver()
app = create_react_agent(
    llm, tools, messages_modifier=system_message, checkpointer=memory
)

In [34]:
config = {"configurable": {"thread_id": "test-thread"}}
print(
    app.invoke(
        {
            "messages": [
                ("user", f"whanyu1212, {GITHUB_ACCESS_TOKEN}. Return the summarized text as well as the frequency table of different types of events."),
            ]
        },
        config,
    )["messages"][-1].content)

### Recent GitHub Activity Summary for User **whanyu1212**

1. **MemberEvent**: Added a new member `Yuxuan1998` to the repository `whanyu1212/Transcendent`.
2. **PushEvent**: Multiple commits to `whanyu1212/Transcendent`:
   - Example on GitHub events summary.
   - Added JSON into `.gitignore`.
   - Added an example on retrieving GitHub events.
   - Minor font changes.
   - Added to the existing to-do list.
   - Added a simple to-do list in `README`.
   - README draft.
   - Setting up pre-commit hooks.
   - Initial dependencies.
3. **PublicEvent**: Made the repository `whanyu1212/LLM-agent-with-Gemini` public.
4. **PullRequestEvent**: Opened and merged a pull request titled "todo list in read me" in the repository `whanyu1212/Transcendent`.
5. **CreateEvent**: Created the branch `hy-dev` in the repository `whanyu1212/Transcendent`.
6. **WatchEvent**: Starred multiple repositories including:
   - `OthersideAI/self-operating-computer`
   - `chiphuyen/machine-learning-systems-design`
   -